In [35]:
'''
This script reads the raw data from SiPM_simulator, filters each pulse with
digital filter and samples it. Filtered and sampled data are saved
'''

%reset
#!/usr/bin/env python
#-*- coding: utf-8 -*-
import sys
import os
import numpy as np
import pandas as pd
from scipy import signal
from random import randint
import matplotlib.pyplot as plt

def main():
    fig = True
    #fig = False
    acquisition_frequency = 1e+11 # Hz
    pad_int = 50000
    cut_off_frequency = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 500]) * 1e+06 # Hz
    sample_frequency = np.array([100, 200, 500, 1000]) * 1e+06 # Hz

    path = "/home/sommema4/git/SiPM_simulator/data/raw/MicroFC60035_EJ-276"
    path_out = "/home/sommema4/git/SiPM_simulator/data/preproc"
    figures = "/home/sommema4/git/SiPM_simulator/data/figure"

    for filename in os.listdir(path):
        file_in = os.path.join(path, filename)
        if filename.endswith(".csv"):
            print(filename)
            sipm, scintillator, energy, particle = strip_filename(filename)
            dir_name = sipm + "_" + scintillator
            dir_out = os.path.join(path_out, dir_name)
            make_directory(dir_out)
            df = read_csv(file_in)
            print(df.shape)
            if (fig == True):
                fig_dir = os.path.join(figures, dir_name)
                make_directory(fig_dir)
                plot_mean_pulse(df, fig_dir, energy, particle)

            #print(nan(df.iloc[:,2::4]))
            df_padded = pad_dataset(df.iloc[:,1:], pad_int)
            #print("df_padded", df_padded)
            for cut_off in cut_off_frequency:
                for sample in sample_frequency:
                    print(cut_off * 1e-06, sample * 1e-06)
                    filename_out = str(cut_off * 1e-06) + "_" + str(sample * 1e-06) + ".csv"
                    file_out = os.path.join(dir_out, filename_out)
                    df_filter = filter_dataset(df_padded, cut_off, acquisition_frequency)
                    #print("df_filter", df_filter)
                    df_sample = sample_dataset(df_filter, sample, acquisition_frequency)
                    #print("df_sample", df_sample)
                    df_info = info_dataset(df_sample, energy, particle)
                    #print("df_info", df_info)
                    #print(df_info.shape)
                    #print(df_info)
                    if os.path.isfile(file_out):
                        df_info.to_csv(file_out, mode='a', header=False) # append dataframe to file
                    else:
                        df_info.to_csv(file_out, header=False)
    return 0
    
def strip_filename(filename):
    base = os.path.basename(filename)
    name = os.path.splitext(base)[0]
    sipm, scintillator, energy, particle = name.split("_")
    return sipm, scintillator, energy, particle

def make_directory(dir_name):
    if not (os.path.isdir(dir_name)):
        try:
            os.makedirs(dir_name)
        except OSError:
            print ("Creation of the directory %s failed" % dir_name)
        else:
            print ("Successfully created the directory %s " % dir_name)

def read_csv(filename):
    # Read data to pandas dataframe and transpose it
    df = pd.read_csv(filename, sep=' ', header=None)
    df = df.iloc[:, :-1]
    df = df.T
    return df

def nan(df):
    for i in df.columns:
        print(i)
        tt = pd.isnull(df[i])
        print(tt[tt==True].index.values)

def plot_mean_pulse(df, fig_dir, energy, particle):
    time = df.iloc[:, 0]
    light = df.iloc[:, 1::4]
    afterpulse = df.iloc[:, 2::4]
    crosstalk = df.iloc[:, 3::4]
    dark_current = df.iloc[:, 4::4]
    total = pd.DataFrame(light.values + crosstalk.values + afterpulse.values + dark_current.values)
    
    light = light.mean(axis=1)
    afterpulse = afterpulse.mean(axis=1)
    crosstalk = crosstalk.mean(axis=1)
    dark_current = dark_current.mean(axis=1)
    total = total.mean(axis=1)

    fig = plt.figure(figsize=(18,9))
    ax = fig.add_subplot(1,1,1)
    ax.plot(1e+09 * time, total, '-', color='k', label='Total')
    ax.plot(1e+09 * time, light, '--', color='r', label='Light component')
    ax.plot(1e+09 * time, afterpulse, '--', color='g', label='Afterpulse')
    ax.plot(1e+09 * time, crosstalk, '--', color='b', label='Optical crosstalk')
    ax.plot(1e+09 * time, dark_current, '--', color='y', label='Dark current')
    ax.set_xlabel('Time [ns]')
    ax.set_ylabel('Current through load resistor [A]')
    ax.set_title(str(energy) + '_' + str(particle))
    ax.set_xlim(0, 1000)
    ax.legend(loc='upper right')
    
    energy = round(float(energy), 2)
    fig_name = str(energy) + '_' + str(particle) + '.png'
    fig_path = os.path.join(fig_dir, fig_name)
    plt.savefig(fig_path)
    plt.close()
    
def pad_dataset(df, pad_int):
    df.columns = np.arange(len(df.columns))
    pad = np.zeros((pad_int, df.shape[1]))
    df_pad = pd.DataFrame(data=pad)
    df_out = df_pad.append(df)
    return df_out

def noise_dataset(df):
    ss

def filter_wrap(x, b, a):
    #x = x.fillna(0)
    return pd.Series(signal.filtfilt(b, a, x))

def filter_dataset(df, cut_off, freq_acq):
    # Construct and apply filter and amplification
    w = 2 * cut_off / freq_acq
    b, a = signal.butter(2, w, analog=False)
    df_out = df.apply(filter_wrap, axis=0, args=(b, a))
    return df_out

def sample_wrap(x, rep):
    start = randint(0, rep-1)
    return x[start::rep]

def sample_dataset(df, sample, freq_acq):
    rep = int(round(freq_acq / sample, 0)) 
    data = df.to_numpy()
    df_out = pd.DataFrame()
    for i in range(df.shape[1]//4):
        start = randint(0, rep-1)
        #print(data[start::rep, 4*i:4*i+4])
        temp = pd.DataFrame(data[start::rep, 4*i:4*i+4])
        if i == 0:
            df_out = temp
        else:
            df_out = pd.concat([df_out, temp], axis=1)
    #df_out[df_out < 1.0e-07] = 0
    df_out.columns = np.arange(len(df_out.columns))
    return df_out

def info_dataset(df, energy, particle):
    temp = np.zeros((2, df.shape[1]))
    info = pd.DataFrame(data=temp)
    info.iloc[0,:] = energy
    info.iloc[1,:] = particle
    df_out = df.append(info)
    df_out = df_out.T
    return df_out
    
main()



Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


MicroFC60035_EJ-276_0.100000_p.csv
(250000, 9)
2
[]
6
[]
None
1.0 100.0
1.0 200.0
1.0 500.0
1.0 1000.0
2.0 100.0
2.0 200.0
2.0 500.0
2.0 1000.0
3.0 100.0
3.0 200.0
3.0 500.0
3.0 1000.0
4.0 100.0
4.0 200.0
4.0 500.0
4.0 1000.0
5.0 100.0
5.0 200.0
5.0 500.0
5.0 1000.0
6.0 100.0
6.0 200.0
6.0 500.0
6.0 1000.0
7.0 100.0
7.0 200.0
7.0 500.0
7.0 1000.0
8.0 100.0
8.0 200.0
8.0 500.0
8.0 1000.0
9.0 100.0
9.0 200.0
9.0 500.0
9.0 1000.0
10.0 100.0


KeyboardInterrupt: 